### requirement.txt
opencv-python 
time
torch
ultralytics
numpy
pandas
collections
filterpy
deep_sort_realtime
supervision
tqdm

In [1]:
import cv2
import torch
import time
import numpy as np
import os
from ultralytics import YOLO


In [2]:


# Define custom path to save the model
custom_path = "models/yolov8n.pt"  # Change this to your preferred location

# Ensure the directory exists
os.makedirs(os.path.dirname(custom_path), exist_ok=True)

# Download and save model
model = YOLO("yolov8n.pt")  # This will download the model if not found


# Move the model to custom path if it's not already there
default_path = os.path.expanduser("~/.cache/ultralytics/assets/yolov8n.pt")
if os.path.exists(default_path) and not os.path.exists(custom_path):
    os.rename(default_path, custom_path)

print(f"Model downloaded and saved to: {custom_path}")


Model downloaded and saved to: models/yolov8n.pt


In [3]:
VEHICLE_CLASSES = [1, 2, 3, 5, 7]  # bicycle, car, motorcycle, bus, truck
CLASS_NAMES = {
    1: 'bicycle',
    2: 'Car',
    3: 'Motorcycle',
    5: 'Bus',
    7: 'Truck'
}

## Speed Detection from static camera and Kalman Filter as object tracker

In [6]:
import os
import cv2
import numpy as np
from collections import defaultdict, deque
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

# Constants
input_video_path = "scooty_1.mp4"  # Change this to your video file
output_dir = "output/Kalman Filter"
output_video_path = os.path.join(output_dir, "scooty_1_kalman.mp4")

# Create output directory if not exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Initialize video capture
cap = cv2.VideoCapture(input_video_path)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Set up VideoWriter
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

# Load YOLO model
model = YOLO("yolov8n.pt")

# Vehicle classes in COCO dataset
VEHICLE_CLASSES = {1, 2, 3, 5, 7}  # Car, Truck, Bus, Motorcycle

# Pixel-to-Meter conversion (adjust based on camera setup)
scale_factor = 0.009  # Example: 1 pixel = 0.05 meters (TUNE THIS)

# Initialize tracker dictionary
trackers = {}
track_id = 0
prev_positions = {}  # Store last known positions
speed_history = defaultdict(lambda: deque(maxlen=5 * fps))  # Store last 5 seconds of speed

def create_kalman_filter():
    """Creates and returns a Kalman filter for tracking."""
    kf = KalmanFilter(dim_x=4, dim_z=2)
    
    # State transition matrix (Position + Velocity Model)
    kf.F = np.array([[1, 0, 1, 0],
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    
    # Measurement matrix (We only measure position)
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 1, 0, 0]])
    
    # Covariance and noise
    kf.P *= 1000  # Large initial uncertainty
    kf.Q = np.eye(4) * 0.1  # Process noise
    kf.R = np.eye(2) * 10  # Measurement noise
    
    return kf

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO Object Detection
    results = model(frame)
    detected_objects = []

    # Process YOLO Detections
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls[0])

        if class_id in VEHICLE_CLASSES:
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            detected_objects.append((center_x, center_y, x1, y1, x2, y2, class_id))

    new_trackers = {}

    for obj in detected_objects:
        center_x, center_y, x1, y1, x2, y2, class_id = obj
        matched = False

        # Compare with existing trackers
        for tid, tracker in trackers.items():
            pred_x, pred_y = tracker.x[:2].flatten()
            dist = np.linalg.norm([pred_x - center_x, pred_y - center_y])

            if dist < 50:  # Threshold for matching
                tracker.predict()  # Predict next position
                tracker.update(np.array([[center_x], [center_y]]))  # Update filter
                new_trackers[tid] = tracker
                matched = True
                break

        # Create a new tracker if no match found
        if not matched:
            track_id += 1
            new_tracker = create_kalman_filter()
            new_tracker.x[:2] = np.array([[center_x], [center_y]]).reshape(2, 1)
            new_trackers[track_id] = new_tracker

    trackers = new_trackers

    # Draw Tracking Results
    for tid, tracker in trackers.items():
        pred_x, pred_y = tracker.x[:2].flatten()

        # Speed Calculation with Smoothing
        if tid in prev_positions:
            prev_x, prev_y = prev_positions[tid]
            distance_px = np.linalg.norm([pred_x - prev_x, pred_y - prev_y])
            distance_m = distance_px * scale_factor  # Convert to meters
            speed_mps = distance_m * fps  # Convert to meters per second
            speed_kph = int(speed_mps * 3.6)  # Convert to km/h and round to integer
        else:
            speed_kph = 0  # First frame

        # Store last position
        prev_positions[tid] = (pred_x, pred_y)

        # Store speed history for smoothing
        speed_history[tid].append(speed_kph)
        smoothed_speed = int(np.mean(speed_history[tid]))  # Averaging last 3 sec speeds

        # Draw tracking centroid
        cv2.circle(frame, (int(pred_x), int(pred_y)), 5, (0, 0, 255), -1)  # Red dot for tracking
        
        # Find the corresponding bounding box
        for obj in detected_objects:
            center_x, center_y, x1, y1, x2, y2, class_id = obj
            if np.linalg.norm([pred_x - center_x, pred_y - center_y]) < 50:
                label = f"{model.names[class_id]} {smoothed_speed}km/h"
                
                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box
                
                # Display ID and speed (Font size 1.3)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                break  # Stop after finding the correct bounding box

    # Write frame to output video
    if out.isOpened():
        out.write(frame)

    # Show video
    cv2.imshow("Vehicle Speed Estimation", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and writer
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Speed-tracked video successfully saved at: {output_video_path}")



0: 288x640 4 persons, 1 car, 37.6ms
Speed: 1.5ms preprocess, 37.6ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 3 persons, 1 car, 37.0ms
Speed: 1.1ms preprocess, 37.0ms inference, 0.5ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 3 persons, 1 car, 45.4ms
Speed: 1.5ms preprocess, 45.4ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 persons, 1 car, 44.4ms
Speed: 1.7ms preprocess, 44.4ms inference, 0.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 persons, 1 car, 45.5ms
Speed: 1.2ms preprocess, 45.5ms inference, 0.8ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 2 persons, 1 car, 43.0ms
Speed: 1.2ms preprocess, 43.0ms inference, 0.4ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 3 persons, 1 car, 42.8ms
Speed: 1.2ms preprocess, 42.8ms inference, 1.7ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 3 persons, 1 car, 43.0ms
Speed: 1.9ms preprocess, 

## Speed Detection from static camera and DeepSort as object tracker

In [ ]:
import os
import cv2
import numpy as np
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO

# Constants
MODEL_NAME = "yolov8n.pt"
INPUT_VIDEO_PATH = "video_4.mp4"
OUTPUT_DIR = "output/DeepSort"
OUTPUT_VIDEO_PATH = os.path.join(OUTPUT_DIR, "Speed_video_4_deepsort.mp4")
PIXEL_TO_METER = 0.05  # 🔹 Convert pixels to meters (Adjust as needed)
SMOOTHING_FRAMES = 5  # 🔹 Smoothing speed over last N frames

# Load YOLO model
model = YOLO(MODEL_NAME)

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=50)

# Ensure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Open video
cap = cv2.VideoCapture(INPUT_VIDEO_PATH)

# Get video properties
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Set up video writer
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(OUTPUT_VIDEO_PATH, fourcc, fps, (frame_width, frame_height))

# Check if video writer is working
if not out.isOpened():
    print("Error: VideoWriter failed to open!")

# Track history & speed calculation
track_history = {}  # Store position history
speed_history = {}  # Store speed history
track_colors = {}  # Assign colors to IDs

def get_color(track_id):
    """Assigns a unique color to each tracking ID"""
    np.random.seed(int(track_id))
    return tuple(np.random.randint(50, 255, 3).tolist())  # Random RGB colors

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop if video ends

    # YOLOv8 Object Detection
    results = model(frame)

    detections = []  # Store detections for DeepSORT

    # Process YOLO Detections
    for box in results[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box coordinates
        class_id = int(box.cls[0])  # Object class ID
        confidence = float(box.conf[0])  # Confidence score

        # Filter only vehicle classes
        VEHICLE_CLASSES = {2, 3, 5, 7}  # Car, Truck, Bus, Motorcycle
        if class_id in VEHICLE_CLASSES:
            detections.append(([x1, y1, x2 - x1, y2 - y1], confidence, class_id))

    # Update DeepSORT Tracker
    tracked_objects = tracker.update_tracks(detections, frame=frame)

    # Process tracked objects
    for track in tracked_objects:
        if track is None or not track.is_confirmed():
            continue  # Ignore unconfirmed tracks

        track_id = int(track.track_id)  # Ensure ID is integer
        x1, y1, w, h = map(int, track.to_ltwh())  # Get bounding box
        center_x, center_y = x1 + w // 2, y1 + h // 2  # Calculate object center

        # Assign color if new ID appears
        if track_id not in track_colors:
            track_colors[track_id] = get_color(track_id)

        # Store tracking history
        if track_id not in track_history:
            track_history[track_id] = []
        track_history[track_id].append((center_x, center_y))

        # Speed Calculation
        if track_id in track_history and len(track_history[track_id]) > 1:
            prev_x, prev_y = track_history[track_id][-2]
            distance_px = np.linalg.norm([center_x - prev_x, center_y - prev_y])  # Pixel distance
            distance_m = distance_px * PIXEL_TO_METER  # Convert to meters
            speed_mps = distance_m * fps  # Convert to meters/second
            speed_kph = int(speed_mps * 3.6)  # Convert to km/hf

            # Smooth speed calculation
            if track_id not in speed_history:
                speed_history[track_id] = []
            speed_history[track_id].append(speed_kph)
            if len(speed_history[track_id]) > SMOOTHING_FRAMES:
                speed_history[track_id].pop(0)  # Keep last N speeds
            smoothed_speed = int(np.mean(speed_history[track_id]))  # Moving average
        else:
            smoothed_speed = 0  # Default speed

        # Draw bounding box
        color = track_colors[track_id]
        cv2.rectangle(frame, (x1, y1), (x1 + w, y1 + h), color, 2)

        # Display Class, ID, and Speed
        label = f"{model.names[class_id]} #{track_id} {smoothed_speed} km/h"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, color, 2)

        # Draw tracking path
        points = track_history[track_id]
        for i in range(1, len(points)):
            cv2.line(frame, points[i - 1], points[i], color, 2)  # Colored tracking line

    # Write frame to output video
    out.write(frame)

    # Show video
    cv2.imshow("Vehicle Tracking with Speed", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Tracked video with speed saved at: {OUTPUT_VIDEO_PATH}")


## Speed Detection from static camera and ByteTrack as object tracker

In [10]:
import cv2
import numpy as np
import supervision as sv
from tqdm import tqdm
from ultralytics import YOLO
from collections import defaultdict, deque

# Constants
MODEL_NAME = "yolov8n.pt"
SOURCE_VIDEO_PATH = "scooty_1.mp4"
TARGET_VIDEO_PATH = "output/scooty_1_bytetrack.mp4"
MODEL_RESOLUTION = 640  
CONFIDENCE_THRESHOLD = 0.4
IOU_THRESHOLD = 0.5  
PIXEL_TO_METER = 0.01  # Convert pixels to meters (Adjust this based on camera)
SMOOTHING_FRAMES = 10  # Smoothing over last N frames

# Load YOLO model
model = YOLO(MODEL_NAME)

# Load video info
video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)
frame_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)

# Initialize ByteTrack Tracker
byte_track = sv.ByteTrack(frame_rate=video_info.fps)

# Annotators Configuration
thickness = max(1, int(min(video_info.resolution_wh) / 500))  
text_scale = max(0.4, min(video_info.resolution_wh) / 1200)  

bounding_box_annotator = sv.BoundingBoxAnnotator(thickness=thickness)
label_annotator = sv.LabelAnnotator(
    text_scale=text_scale,
    text_thickness=thickness,
    text_position=sv.Position.TOP_LEFT
)
trace_annotator = sv.TraceAnnotator(
    thickness=thickness,
    trace_length=video_info.fps * 10,  
    position=sv.Position.BOTTOM_CENTER
)

# Tracking History
coordinates = defaultdict(lambda: deque(maxlen=video_info.fps))  # Store last second of positions
speed_history = defaultdict(lambda: deque(maxlen=SMOOTHING_FRAMES))  # Store last N speeds for smoothing

# Map YOLO class IDs to names
class_names = model.names  

with sv.VideoSink(TARGET_VIDEO_PATH, video_info) as sink:
    for frame in tqdm(frame_generator, total=video_info.total_frames):
        
        # Rotate frame by 180 degrees
        # frame = cv2.rotate(frame, cv2.ROTATE_180)
        
        # Object Detection with YOLO
        result = model(frame, imgsz=MODEL_RESOLUTION, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(result)

        # Filter detections by confidence
        detections = detections[detections.confidence > CONFIDENCE_THRESHOLD]

        # Filter only vehicle classes
        vehicle_classes = {2, 3, 5, 7}  # Car, Truck, Bus, Motorcycle
        detections = detections[np.isin(detections.class_id, list(vehicle_classes))]

        # Apply Non-Maximum Suppression (NMS)
        detections = detections.with_nms(IOU_THRESHOLD)

        # Track Objects
        detections = byte_track.update_with_detections(detections)

        # Speed Calculation
        vehicle_speeds = {}  
        for tracker_id, (x, y) in zip(detections.tracker_id, detections.xyxy[:, :2]):
            x, y = int(x), int(y)  # Convert to integer

            # Save position history
            if tracker_id in coordinates:
                prev_x, prev_y = coordinates[tracker_id][-1]
                distance_px = np.linalg.norm([x - prev_x, y - prev_y])
                distance_m = distance_px * PIXEL_TO_METER  
                speed_mps = distance_m * video_info.fps  
                speed_kph = int(speed_mps * 3.6)  

                # Store speed for smoothing
                speed_history[tracker_id].append(speed_kph)
                smoothed_speed = int(np.mean(speed_history[tracker_id]))  
            else:
                smoothed_speed = 0  

            # Save updated position
            coordinates[tracker_id].append((x, y))
            vehicle_speeds[tracker_id] = smoothed_speed  

        # Format Labels (Class Name + Tracker ID + Speed)
        labels = [f"{class_names[class_id]} {vehicle_speeds.get(tracker_id, 0)}km/h" 
                  for class_id, tracker_id in zip(detections.class_id, detections.tracker_id)]

        # Annotate the Frame
        annotated_frame = frame.copy()
        annotated_frame = trace_annotator.annotate(scene=annotated_frame, detections=detections)
        annotated_frame = bounding_box_annotator.annotate(scene=annotated_frame, detections=detections)
        annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)

        # Save the frame to output video
        sink.write_frame(annotated_frame)

print(f"✅ Tracking complete. Output saved as: {TARGET_VIDEO_PATH}")


SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
100%|█████████████████████████████████████████| 525/525 [00:20<00:00, 25.11it/s]

✅ Tracking complete. Output saved as: output/scooty_1_bytetrack.mp4


In [ ]:
## Another Model

In [19]:
import numpy as np
import cv2
import torch
from ultralytics import YOLO
import math
import time

# Define vehicle classes from COCO dataset
VEHICLE_CLASSES = [1, 2, 3, 5, 7]  # Car, Motorcycle, Bus, Truck
CLASS_NAMES = {1:'bicycle', 2: 'Car', 3: 'Motorcycle', 5: 'Bus', 7: 'Truck'}

class MotionCompensator:
    def __init__(self):
        self.feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
        self.lk_params = dict(winSize=(15, 15), maxLevel=2, 
                              criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
        self.prev_gray = None
        self.prev_points = None
        
    def estimate_camera_motion(self, frame):
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        if self.prev_gray is None:
            self.prev_gray = gray
            self.prev_points = cv2.goodFeaturesToTrack(gray, mask=None, **self.feature_params)
            return np.eye(3)  # Identity matrix for first frame

        if self.prev_points is None or len(self.prev_points) < 10:
            self.prev_points = cv2.goodFeaturesToTrack(self.prev_gray, mask=None, **self.feature_params)
            if self.prev_points is None:
                self.prev_gray = gray
                return np.eye(3)

        next_points, status, _ = cv2.calcOpticalFlowPyrLK(self.prev_gray, gray, self.prev_points, None, **self.lk_params)
        if next_points is None or len(next_points) < 4:
            self.prev_gray = gray
            return np.eye(3)

        good_old = self.prev_points[status == 1]
        good_new = next_points[status == 1]

        if len(good_old) < 4 or len(good_new) < 4:
            self.prev_gray = gray
            self.prev_points = next_points
            return np.eye(3)

        transform_matrix, _ = cv2.estimateAffinePartial2D(good_old, good_new)
        if transform_matrix is None:
            transform_matrix = np.eye(3)[:2]

        full_transform = np.eye(3)
        full_transform[:2] = transform_matrix
        self.prev_gray = gray
        self.prev_points = good_new.reshape(-1, 1, 2)
        return full_transform

class SpeedEstimator:
    def __init__(self, camera_fps=30, px_to_meter_ratio=10):
        self.fps = camera_fps
        self.px_to_meter_ratio = px_to_meter_ratio
        self.prev_positions = {}
        self.prev_times = {}
        self.speed_buffer = {}
        self.buffer_size = 5

    def calculate_speed(self, track_id, current_pos, current_time, camera_motion):
        if track_id not in self.prev_positions:
            self.prev_positions[track_id] = current_pos
            self.prev_times[track_id] = current_time
            self.speed_buffer[track_id] = []
            return None

        prev_pos = self.prev_positions[track_id]
        prev_time = self.prev_times[track_id]
        time_diff = current_time - prev_time

        if time_diff <= 0:
            return None

        adjusted_prev_pos = self.apply_transform(prev_pos, camera_motion)
        displacement = np.linalg.norm(current_pos - adjusted_prev_pos)
        displacement_meters = displacement / self.px_to_meter_ratio
        speed_ms = displacement_meters / time_diff
        speed_kmh = speed_ms * 3.6

        self.speed_buffer[track_id].append(speed_kmh)
        if len(self.speed_buffer[track_id]) > self.buffer_size:
            self.speed_buffer[track_id].pop(0)

        filtered_speed = np.mean(self.speed_buffer[track_id])
        self.prev_positions[track_id] = current_pos
        self.prev_times[track_id] = current_time
        return filtered_speed

    def apply_transform(self, point, transform):
        homog_point = np.array([point[0], point[1], 1])
        transformed = transform @ homog_point
        return transformed[:2] / transformed[2]

class VehicleTracker:
    def __init__(self, video_path, yolo_model='yolov8n.pt', conf_thresh=0.5):
        self.model = YOLO(yolo_model)
        self.conf_thresh = conf_thresh
        self.cap = cv2.VideoCapture(video_path)
        self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)

        self.motion_compensator = MotionCompensator()
        self.speed_estimator = SpeedEstimator(camera_fps=self.fps)

        self.output_path = 'output/scooty_he_' + video_path.split('/')[-1]
        self.writer = cv2.VideoWriter(self.output_path, cv2.VideoWriter_fourcc(*'mp4v'),
                                      self.fps, (self.frame_width, self.frame_height))

        self.tracks = {}
        self.next_id = 0

        # Annotator Parameters
        self.thickness = max(1, int(min(self.frame_width, self.frame_height) / 500))
        self.text_scale = max(0.4, min(self.frame_width, self.frame_height) / 1200)

    def get_track_id(self, bbox, max_dist=50):
        center = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2])
        for track_id, track_info in self.tracks.items():
            if np.linalg.norm(center - track_info['center']) < max_dist:
                return track_id
        self.next_id += 1
        return self.next_id

    def process_video(self):
        while True:
            ret, frame = self.cap.read()
            # frame = cv2.rotate(frame, cv2.ROTATE_180)
            if not ret:
                break

            camera_motion = self.motion_compensator.estimate_camera_motion(frame)
            current_time = time.time()
            results = self.model(frame, conf=self.conf_thresh, classes=VEHICLE_CLASSES, verbose=False)[0]

            active_tracks = set()

            for bbox, cls in zip(results.boxes.xyxy, results.boxes.cls):
                bbox = bbox.cpu().numpy()
                cls = int(cls)
                center = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2])
                track_id = self.get_track_id(bbox)
                active_tracks.add(track_id)
                self.tracks[track_id] = {'center': center, 'class': cls, 'bbox': bbox}

                speed = self.speed_estimator.calculate_speed(track_id, center, current_time, camera_motion)

                # Draw Bounding Box
                cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])),
                              (0, 255, 0), self.thickness)

                # Format Label
                label = f"{CLASS_NAMES.get(cls, 'Vehicle')} {speed:.1f}km/h" if speed is not None else CLASS_NAMES.get(cls, 'Vehicle')

                # Put Label (Top-Left of Bounding Box)
                text_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, self.text_scale, self.thickness)[0]
                label_bg_coords = (int(bbox[0]), int(bbox[1] - text_size[1] - 5))
                label_text_coords = (int(bbox[0]), int(bbox[1] - 5))
                
                cv2.rectangle(frame, label_bg_coords, (int(bbox[0] + text_size[0]), int(bbox[1])), (0, 255, 0), -1)
                cv2.putText(frame, label, label_text_coords, cv2.FONT_HERSHEY_SIMPLEX, self.text_scale, (0, 0, 0), self.thickness)

            self.tracks = {k: v for k, v in self.tracks.items() if k in active_tracks}
            self.writer.write(frame)
            cv2.imshow('Vehicle Tracking and Speed Estimation', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        self.cap.release()
        self.writer.release()
        cv2.destroyAllWindows()

def main():
    tracker = VehicleTracker("scooty_2.mp4")
    tracker.process_video()

if __name__ == "__main__":
    main()


In [10]:
# import numpy as np
# import cv2
# import torch
# from ultralytics import YOLO
# import math
# import time
# from filterpy.kalman import KalmanFilter

# # Define vehicle classes from COCO dataset
# VEHICLE_CLASSES = [2, 3, 5, 7]  # car, motorcycle, bus, truck
# CLASS_NAMES = {
#     2: 'Car',
#     3: 'Motorcycle',
#     5: 'Bus',
#     7: 'Truck'
# }

# class KalmanTracker:
#     def __init__(self):
#         self.kf = KalmanFilter(dim_x=4, dim_z=2)
#         self.kf.F = np.array([[1, 0, 1, 0],
#                               [0, 1, 0, 1],
#                               [0, 0, 1, 0],
#                               [0, 0, 0, 1]])
#         self.kf.H = np.array([[1, 0, 0, 0],
#                               [0, 1, 0, 0]])
#         self.kf.P *= 1000
#         self.kf.R = np.array([[1, 0], [0, 1]])
#         self.kf.Q = np.eye(4) * 0.01
#         self.initialized = False
    
#     def update(self, measurement):
#         if not self.initialized:
#             self.kf.x[:2] = measurement.reshape(-1, 1)
#             self.initialized = True
#         self.kf.predict()
#         self.kf.update(measurement)
#         return self.kf.x[:2].flatten()

# class VehicleTracker:
#     def __init__(self, video_path, yolo_model='yolov8n.pt', conf_thresh=0.5):
#         self.model = YOLO(yolo_model)
#         self.conf_thresh = conf_thresh
#         self.cap = cv2.VideoCapture(video_path)
#         self.frame_width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
#         self.frame_height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
#         self.fps = self.cap.get(cv2.CAP_PROP_FPS)
#         self.output_path = 'output/tracked_video.mp4'
#         self.writer = cv2.VideoWriter(
#             self.output_path,
#             cv2.VideoWriter_fourcc(*'mp4v'),
#             self.fps,
#             (self.frame_width, self.frame_height)
#         )
#         self.tracks = {}
#         self.next_id = 0
#         self.px_to_meter_ratio = 10

#     def get_track_id(self, bbox, max_dist=50):
#         center = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2])
#         for track_id, tracker in self.tracks.items():
#             track_center = tracker['tracker'].kf.x[:2]
#             if np.linalg.norm(center - track_center) < max_dist:
#                 return track_id
#         self.next_id += 1
#         return self.next_id

#     def calculate_speed(self, track_id, current_pos, current_time):
#         if 'prev_time' not in self.tracks[track_id]:
#             self.tracks[track_id]['prev_pos'] = current_pos
#             self.tracks[track_id]['prev_time'] = current_time
#             return None
#         prev_pos = self.tracks[track_id]['prev_pos']
#         prev_time = self.tracks[track_id]['prev_time']
#         time_diff = current_time - prev_time
#         if time_diff <= 0:
#             return None
#         displacement = np.linalg.norm(current_pos - prev_pos) / self.px_to_meter_ratio
#         speed_kmh = (displacement / time_diff) * 3.6
#         self.tracks[track_id]['prev_pos'] = current_pos
#         self.tracks[track_id]['prev_time'] = current_time
#         return speed_kmh

#     def process_video(self):
#         while True:
#             ret, frame = self.cap.read()
#             frame = cv2.rotate(frame, cv2.ROTATE_180)
#             if not ret:
#                 break
#             current_time = time.time()
#             results = self.model(frame, conf=self.conf_thresh, classes=VEHICLE_CLASSES, verbose=False)[0]
#             active_tracks = set()
#             for bbox, cls in zip(results.boxes.xyxy, results.boxes.cls):
#                 bbox = bbox.cpu().numpy()
#                 cls = int(cls)
#                 center = np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2])
#                 track_id = self.get_track_id(bbox)
#                 active_tracks.add(track_id)
#                 if track_id not in self.tracks:
#                     self.tracks[track_id] = {'tracker': KalmanTracker(), 'class': cls}
#                 smoothed_center = self.tracks[track_id]['tracker'].update(center)
#                 speed = self.calculate_speed(track_id, smoothed_center, current_time)
#                 cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (0, 255, 0), 2)
#                 label = f"{CLASS_NAMES.get(cls, 'Vehicle')} {speed:.1f} km/h" if speed else CLASS_NAMES.get(cls, 'Vehicle')
#                 cv2.putText(frame, label, (int(bbox[0]), int(bbox[1]) + 25),
#                             cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
#             self.tracks = {k: v for k, v in self.tracks.items() if k in active_tracks}
#             self.writer.write(frame)
#             cv2.imshow('Vehicle Tracking and Speed Estimation', frame)
#             if cv2.waitKey(1) & 0xFF == ord('q'):
#                 break
#         self.cap.release()
#         self.writer.release()
#         cv2.destroyAllWindows()

# def main():
#     video_path = "input/Roorkee_4.mp4"
#     tracker = VehicleTracker(video_path, yolo_model='yolov8n.pt', conf_thresh=0.5)
#     tracker.process_video()

# if __name__ == "__main__":
#     main()
